<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-project/blob/main/BLM6114_project_tr_llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Kütüphaneler

In [ ]:
!pip install -q torch accelerate transformers datasets huggingface-hub

## Modellere göre sınıflandırma (Çözebilir mi?)

Modeller:

https://huggingface.co/ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1



### Modelin yüklenmesi

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import notebook_login

# pipeline'ları oluşturma
tokenizer = AutoTokenizer.from_pretrained("ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1")
turkish_llama = AutoModelForCausalLM.from_pretrained(
    "ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
    torch_dtype=torch.float16,
    device_map="cuda",
)

turkish_llama.eval()

#turkish_llama = pipeline(
#    "text-generation",
#    model="ytu-ce-cosmos/Turkish-Llama-8b-DPO-v0.1",
#    model_kwargs={"torch_dtype": torch.bfloat16},
#    device_map="cuda",
#)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps

### Değerlendirme Fonksiyonu

In [ ]:
import re

def extract_number(text):
    # Metindeki sayıları bul, sonuncusunu float’a çevir. Son sayı cevap.
    nums = re.findall(r"[-+]?\d+\.?\d*", text.replace(",","."))
    return float(nums[-1]) if nums else None

def predict_tr_llama(question):
  prompt = (
    f"Soru: {question}\n"
    "Soruya tek bir yanıt ver, adım adım çözme, sadece sonuç olsun.",
    "Yanıtlarında ‘24 \times \frac’ gibi notasyonlar kullanma.",
    "Bunu da ‘Sonuç: <değer>’ formatında göster."
  )

  messages = [
    {"role": "user", "content": prompt},
  ]

  input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
  ).to(turkish_llama.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = turkish_llama.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
  )

  response = outputs[0][input_ids.shape[-1]:]
  assistant_response = tokenizer.decode(response, skip_special_tokens=True)
  return assistant_response

## Not working as expected!!!
def predict_tr_llama_multi(questions):
  messages = []
  for question in questions:
    prompt = (
      f"Soru: {question}\n"
      "Soruya kısa ve net bir yanıt ver, adım adım çözme, yanıtlarında ‘24 \times \frac’ gibi notasyonlar kullanma, bunu da ‘Sonuç: <değer>’ formatında göster."
    )
    messages.append({"role": "user", "content": prompt})

  terminators = [
    turkish_llama.tokenizer.eos_token_id,
    turkish_llama.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  print(f"Mesages --> {messages}");

  outputs = turkish_llama(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
  )

  print(f"Outputs --> {outputs}");

  results = []
  for output in outputs:
    assistant_response = output["generated_text"][-1]["content"].strip()
    results.append(assistant_response)

  return results


def label_prediction(question, true_answer):
  tr_llama_result = predict_tr_llama(question)
  tr_llama_prediction = extract_number(tr_llama_result)
  real = extract_number(true_answer)
  # Tahmin gerçek değerle eşleşiyorsa 1, yoksa 0
  is_tr_llama_correct = 1 if (tr_llama_prediction is not None and real is not None and abs(tr_llama_prediction-real)<1e-3) else 0
  return tr_llama_result, is_tr_llama_correct

### Örnek Soru-Cevap Karşılaştırması

In [ ]:
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "info"

df = pd.read_csv("gsm8k_tr_classified.csv")

results = []

# Sadece ilk 10 kayıt için döngü
for i in range(10):
  q = df.question.iloc[i]
  true_ans = df.answer.iloc[i]

  tr_llama_result, is_tr_llama_correct = label_prediction(q, true_ans)

  # DataFrame'e yeni sütunlar ekleyelim
  df.loc[i, "tr_llama_result"] = tr_llama_result
  df.loc[i, "is_tr_llama_correct"] = is_tr_llama_correct

# Sonuçları DataFrame olarak görüntüle (İlk 10 satır için)
print(df[:10])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a conseque

                                            question  \
0  Borris tekel bayisi her 6 ayda bir 90 kilogram...   
1  Mel, Katherine'den üç yaş küçük.  Katherine ik...   
2  James 2 ağacındaki tüm meyveleri toplar.  Her ...   
3  Kyle, her biri 15 origami yıldızı alabilen 2 c...   
4  Mark'ın iki evcil hayvanı var: Saniyede 10 adı...   
5  Sinema biletlerinin tanesi Pazartesi günü 5 do...   
6  Conner'ın çölde sürdüğü bir kumul arabası var....   
7  Bir mağaza her hafta 100 gramlık 20 paket şeke...   
8  James arabasını değiştirmeye karar verir.  20....   
9  Simon, Anneler Günü için annesinin dikebileceğ...   

                                              answer  \
0  Borris şu anda her 6 ayda 90 kilogram üzüm kul...   
1  Katherine iki düzine yaşına geldiğinde 24 yaşı...   
2                           James 24 ağaç dikmiştir.   
3  Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...   
4  Kaplumbağanın yarışı berabere bitirmesi için 1...   
5  Glenn Çarşamba günü 10 dolar, Cumartesi günü

In [28]:
from tqdm.notebook import tqdm
import os

os.environ["TRANSFORMERS_VERBOSITY"] = "info"


# Tüm DataFrame üzerinde döngü
for i in tqdm(range(len(df)), desc="Processing rows"):
  q = df.question.iloc[i]
  true_ans = df.answer.iloc[i]

  tr_llama_result, is_tr_llama_correct = label_prediction(q, true_ans)

  # DataFrame'e yeni sütunları ekleyelim
  df.loc[i, "tr_llama_result"] = tr_llama_result
  df.loc[i, "is_tr_llama_correct"] = is_tr_llama_correct

  # Her 100 iterasyonda bir veya döngünün sonunda kaydet
  if (i + 1) % 100 == 0 or (i + 1) == len(df):
    df.to_csv("gsm8k_tr_classified_with_llama_results.csv", index=False)
    print(f"{i+1} kayıt işlendi ve kaydedildi.")


print("Sonuçlar 'gsm8k_tr_classified_with_llama_results.csv' dosyasına kaydedildi.")

Processing rows:   0%|          | 0/8792 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a conseque

100 kayıt işlendi ve kaydedildi.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a conseque

200 kayıt işlendi ve kaydedildi.


...

8600 kayıt işlendi ve kaydedildi.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a conseque

8700 kayıt işlendi ve kaydedildi.


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a conseque

8792 kayıt işlendi ve kaydedildi.
Sonuçlar 'gsm8k_tr_classified_with_llama_results.csv' dosyasına kaydedildi.


In [29]:
# prompt: sonuç dosyasının analizini çıkarır mısın?

# Calculate the accuracy of the Gemma model
accuracy = df['is_tr_llama_correct'].mean() * 100
print(f"LLama Model Doğruluğu: {accuracy:.2f}%")

# Display some examples where the model was incorrect
incorrect_predictions = df[df['is_tr_llama_correct'] == 0]
print("\nYanlış Tahmin Edilen Örnekler:")
display(incorrect_predictions[['question', 'answer', 'tr_llama_result']].head())

# You could also analyze the distribution of correct vs incorrect predictions
correct_counts = df['is_tr_llama_correct'].value_counts()
print("\nDoğru/Yanlış Tahmin Dağılımı:")
correct_counts


LLama Model Doğruluğu: 11.39%

Yanlış Tahmin Edilen Örnekler:


,question,answer,tr_llama_result
0,Borris tekel bayisi her 6 ayda bir 90 kilogram...,Borris şu anda her 6 ayda 90 kilogram üzüm kul...,Sonuç: 108000
1,"Mel, Katherine'den üç yaş küçük. Katherine ik...",Katherine iki düzine yaşına geldiğinde 24 yaşı...,Sonuç: 20
2,James 2 ağacındaki tüm meyveleri toplar. Her ...,James 24 ağaç dikmiştir.,Sonuç: 8
3,"Kyle, her biri 15 origami yıldızı alabilen 2 c...",Kyle toplamda 5 cam şişe satın aldı (2 + 3). H...,Sonuç: 360
5,Sinema biletlerinin tanesi Pazartesi günü 5 do...,"Glenn Çarşamba günü 10 dolar, Cumartesi günü i...",Sonuç: 55 $



Doğru/Yanlış Tahmin Dağılımı:


,count
is_tr_llama_correct,
0.0,7791
1.0,1001
